In [ ]:
#| default_exp spec

In [ ]:
#| export
from fastcore.all import *

import os, pprint, re

In [ ]:
stripe_openapi_url = 'https://raw.githubusercontent.com/stripe/openapi/refs/heads/master/openapi/spec3.json'
stripe_spec = urlsend(stripe_openapi_url, 'GET', return_json=True)
stripe_spec.keys()

dict_keys(['components', 'info', 'openapi', 'paths', 'security', 'servers'])

The OpenAPI spec describes how a particular REST API works. The most important part of this spec is the paths that are defined by a particular spec. This defines what you can do with a particular API and usually includes things like the description of endpoint, expected parameters, the description of the parameters, and a schema of what will be returned from a particular endpoint.

In [ ]:
path, verbs = first(stripe_spec['paths'].items())
path, verbs.keys()

('/v1/account', dict_keys(['get']))

In [ ]:
verbs['get'].keys()

dict_keys(['description', 'operationId', 'parameters', 'requestBody', 'responses', 'summary'])

As we can see here, the account path has a single HTTP verb that we can use on it called GET. The verb + path is what we refer to as an endpoint. This endpoint allows us to get the account details for a stripe.

In [ ]:
stripe_api_url = 'https://api.stripe.com'
ep = stripe_api_url + path
ep

'https://api.stripe.com/v1/account'

In [ ]:
stripe_key = os.environ['STRIPE_SECRET_KEY']
headers = {'Authorization': f'Bearer {stripe_key}'}
resp = urlsend(ep, 'GET', headers=headers)
resp.keys()

dict_keys(['id', 'object', 'business_type', 'capabilities', 'charges_enabled', 'country', 'default_currency', 'details_submitted', 'payouts_enabled', 'settings', 'type'])

Some of these endpoints will take parameters if they are GET verbs or request bodies. POST verbs. Here is an example for the GET customers endpoint

In [ ]:
path, verbs = first(stripe_spec['paths'].items(), lambda x: x[0] == '/v1/customers')
verbs['get']['parameters'][0]

{'description': 'Only return customers that were created during the given date interval.',
 'explode': True,
 'in': 'query',
 'name': 'created',
 'required': False,
 'schema': {'anyOf': [{'properties': {'gt': {'type': 'integer'},
     'gte': {'type': 'integer'},
     'lt': {'type': 'integer'},
     'lte': {'type': 'integer'}},
    'title': 'range_query_specs',
    'type': 'object'},
   {'type': 'integer'}]},
 'style': 'deepObject'}

And here is one for a post

In [ ]:
first(nested_idx(verbs, *'post requestBody content application/x-www-form-urlencoded schema properties'.split()).items())

('address',
 {'anyOf': [{'properties': {'city': {'maxLength': 5000, 'type': 'string'},
     'country': {'maxLength': 5000, 'type': 'string'},
     'line1': {'maxLength': 5000, 'type': 'string'},
     'line2': {'maxLength': 5000, 'type': 'string'},
     'postal_code': {'maxLength': 5000, 'type': 'string'},
     'state': {'maxLength': 5000, 'type': 'string'}},
    'title': 'optional_fields_customer_address',
    'type': 'object'},
   {'enum': [''], 'type': 'string'}],
  'description': "The customer's address."})

Depending on the spec, the encoding might be json or form url encoded. For Stripe, form url encoded is used.

Finally, parameters can be inside the path itself

In [ ]:
path, verbs = first(stripe_spec['paths'].items(), lambda x: '{' in x[0])
path, verbs.keys()

('/v1/accounts/{account}', dict_keys(['delete', 'get', 'post']))

`fastcore` luckily has some pretty nifty helper functions to deal with path parameters

In [ ]:
path, *_ = partial_format(path)
route_ps = stringfmt_names(path)
route_ps

['account']

For post parameters, we can get data type information that we can surface to developers.

In [ ]:
#| export
_lu_type = dict(zip(
    'NA string object array boolean number integer'.split(),
    map(PrettyString,'object str dict list bool int int'.split())
))

In [ ]:
_lu_type

{'NA': object,
 'string': str,
 'object': dict,
 'array': list,
 'boolean': bool,
 'number': int,
 'integer': int}

In [ ]:
#| export
def _find_data(v, encoding='application/json'):
    'Finds the properties in a schema'
    schema = nested_idx(v, *f'requestBody content {encoding} schema'.split())
    if not schema: return {}
    if 'properties' in schema: return schema['properties']
    if 'oneOf' in schema:
        for o in schema['oneOf']:
            if 'properties' in o: return o['properties']
    return {}

In [ ]:
d = first(_find_data(verbs['post'], 'application/x-www-form-urlencoded').items())
d

('account_token',
 {'description': 'An [account token](https://stripe.com/docs/api#create_account_token), used to securely provide details to the account.',
  'maxLength': 5000,
  'type': 'string'})

In [ ]:
#| export
def _deets(k,v):
    'Extracts the type and default value from a schema'
    return {'name': k, 'description': v.get('description', ''),
        'annotation': _lu_type[v.get('type', 'NA')], 'default' : v.get('default', None)}

In [ ]:
deets = _deets(*d)
deets

{'name': 'account_token',
 'description': 'An [account token](https://stripe.com/docs/api#create_account_token), used to securely provide details to the account.',
 'annotation': str,
 'default': None}

While get requests can technically also have bodies, these are normally ignored by HTTP servers.

The data we care most about for interfacing with an OpenAPI spec are the data, operation id, query parameters, summary, and url. So, lets make a helper function to grab these tidbits

In [ ]:
#| export
def _info(desc):
    # handle both json and form-urlencoded
    data = _find_data(desc, 'application/json') | _find_data(desc, 'application/x-www-form-urlencoded')
    data = [_deets(*o) for o in data.items()]

    params = desc.get('parameters',None)
    qparams = [{'name': p['name'], 'description': p.get('description', ''), 'annotation': 'str'}
               for p in params if p.get('in')=='query'] if params else []
    return {'data': data, 'op_id': desc.get('operationId',''),
            'qparams': qparams, 'summary': desc.get('summary','')}

In [ ]:
info = _info(verbs['get'])
first(info['data']), info['op_id'], first(info['qparams']), info['summary']

(None,
 'GetAccountsAccount',
 {'name': 'expand',
  'description': 'Specifies which fields in the response should be expanded.',
  'annotation': 'str'},
 'Retrieve account')

In [ ]:
info = _info(verbs['post'])
first(info['data']), info['op_id'], first(info['qparams']), info['summary']

({'name': 'account_token',
  'description': 'An [account token](https://stripe.com/docs/api#create_account_token), used to securely provide details to the account.',
  'annotation': str,
  'default': None},
 'PostAccountsAccount',
 None,
 'Update an account')

In [ ]:
#| export
docs_url = 'https://docs.stripe.com/api'

In [ ]:
#| export
def _slug(has_pparams, is_plural, verb):
    'Return the slug for a verb'
    match (has_pparams, is_plural, verb):
        case (_, False, 'get'): return 'retrieve'
        case (True, _, 'post'): return 'update'
        case (False, _, 'get'): return 'list'
        case (False, _, 'post'): return 'create'
        case _: return 'delete'

In [ ]:
#| export
pat = r'/\{[^}]+\}'
def _durl(path, verb):
    'Return the doc url for a path and verb if it exists else None'
    has_pparams = '{' in path
    p = re.sub(pat, '', path)
    is_plural = p.endswith('s')
    if not is_plural: p += 's'
    res = p.split('/')[-1]
    slug = _slug(has_pparams, is_plural, verb)
    return f'{docs_url}/{res}/{slug}'

In [ ]:
_durl(path, 'get')

'https://docs.stripe.com/api/accounts/delete'

In [ ]:
#| export
def build_eps(url):
    "Build module metadata.py from an Open API spec and optionally filter by a path `pre`"
    spec = urlsend(url, 'GET', return_json=True)
    _funcs = [{'path': p, 'verb': v, **_info(desc), 'doc_url': _durl(p, v)}
              for p, vs in spec['paths'].items() for v, desc in vs.items()]
    return _funcs

In [ ]:
eps = build_eps(stripe_openapi_url)
eps[0]

{'path': '/v1/account',
 'verb': 'get',
 'data': [],
 'op_id': 'GetAccount',
 'qparams': [{'name': 'expand',
   'description': 'Specifies which fields in the response should be expanded.',
   'annotation': 'str'}],
 'summary': 'Retrieve account',
 'doc_url': 'https://docs.stripe.com/api/accounts/retrieve'}

In [ ]:
#| export
@call_parse
def update_version():
    'Update the version to the latest version of the Stripe API and the endpoints file.'
    cfg = Config.find("settings.ini")
    stripe_spec = urlsend(stripe_openapi_url, 'GET', return_json=True)
    stripe_version = stripe_spec['info']['version'].split('.')[0].replace('-', '.')

    if cfg.d['version'] == stripe_version: return
    cfg.d['version'] = stripe_version + '.0'
    cfg.save()
    eps = build_eps(stripe_openapi_url)
    (cfg.config_path/'faststripe/endpoints.py').write_text(f'eps = {pprint.pformat(eps, width=360)}')
    print(f"Updated version to {cfg.d['version']}")

In [ ]:
update_version()

Updated version to 2025.11.17.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()